# Ballanced dataset model implementation

    
### Boosting alghoritms: 
    XGBoost
    AdaBoost
    Gradient Boosting Machine
    Optimized light gradient boosting
    CatBoost

## Ballanced dataset Models

In [2]:
# Importing necessary libraries

import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('float_format', '{:f}'.format)

import matplotlib.pyplot as plt
import matplotlib.colors as colors  
import matplotlib.patches as mpatches
%matplotlib inline
import seaborn as sns

import time

import warnings
warnings.filterwarnings('ignore')


from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, recall_score, precision_score

In [3]:
df = pd.read_csv('datasets/data2_balanced.csv')

In [4]:
df.head()

,s_amount,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,0.496764,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,0.637735,-0.987020,0.293438,-0.941386,0.549020,1.804879,0.215598,0.512307,0.333644,0.124270,0.091202,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,0
1,-0.458942,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,0.529808,0.140107,1.564246,0.574074,0.627719,0.706121,0.789188,0.403810,0.201799,-0.340687,-0.233984,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,0
2,-0.794279,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,0.690708,-0.272985,0.659201,0.805173,0.616874,3.069025,-0.577514,0.886526,0.239442,-2.366079,0.361652,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,0
3,-0.554667,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,0.575231,-0.752581,0.737483,0.592994,0.559535,-0.697664,-0.030669,0.242629,2.178616,-1.345060,-0.378223,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,0
4,0.187692,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,0.968046,-1.203171,1.029577,1.439310,0.241454,0.153008,0.224538,0.366466,0.291782,0.445317,0.247237,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,0


In [5]:
# Spliting Training and testing data
X = df.drop('Class', axis=1)
y = df['Class']


In [6]:
sss = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

In [7]:
for train_index, test_index in sss.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    original_Xtrain, original_Xtest = X.iloc[train_index], X.iloc[test_index]
    original_ytrain, original_ytest = y.iloc[train_index], y.iloc[test_index]

# Check the Distribution of the labels
original_Xtrain = original_Xtrain.values
original_Xtest = original_Xtest.values
original_ytrain = original_ytrain.values
original_ytest = original_ytest.values

# Check to see if the distribution of the test and train labels is similar.
train_unique_label, train_counts_label = np.unique(original_ytrain, return_counts=True)
test_unique_label, test_counts_label = np.unique(original_ytest, return_counts=True)
print('-' * 100)

print('Label Distributions: \n')
print(train_counts_label/ len(original_ytrain))
print(test_counts_label/ len(original_ytest))

Train: [ 57020  57021  57022 ... 568627 568628 568629] Test: [     0      1      2 ... 341175 341176 341177]
Train: [     0      1      2 ... 568627 568628 568629] Test: [ 57020  57021  57022 ... 398038 398039 398040]
Train: [     0      1      2 ... 568627 568628 568629] Test: [113967 113968 113969 ... 454901 454902 454903]
Train: [     0      1      2 ... 568627 568628 568629] Test: [170949 170950 170951 ... 511764 511765 511766]
Train: [     0      1      2 ... 511764 511765 511766] Test: [227869 227870 227871 ... 568627 568628 568629]
----------------------------------------------------------------------------------------------------
Label Distributions: 

[0.5 0.5]
[0.5 0.5]


In [8]:
# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

## XGBoost

by including tweaking for max_depth and n_estimators alongside learning_rate, we'll set up a nested loop to iterate over combinations of these parameters. 

In [10]:
import time
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# Define ranges for the parameters to test
learning_rates = [0.01, 0.1, 0.3]
max_depths = [3, 6, 9]
n_estimators = [20, 50, 100, 200, 500, 1000]

# Initialize a list to store the results
results = []

# Loop over the combinations of learning_rates, max_depths, and n_estimators
for lr in learning_rates:
    for depth in max_depths:
        for n_est in n_estimators:
            start_time = time.time()  # Start timing
            
            # Instantiate and fit the XGBoost classifier
            xgb_classifier = xgb.XGBClassifier(learning_rate=lr, max_depth=depth, n_estimators=n_est,
                                               use_label_encoder=False, eval_metric='logloss')
            xgb_classifier.fit(X_train, y_train)
            
            # Predict and evaluate on training data
            y_train_pred = xgb_classifier.predict(X_train)
            train_accuracy = accuracy_score(y_train, y_train_pred)
            train_f1 = f1_score(y_train, y_train_pred)
            train_recall = recall_score(y_train, y_train_pred)
            train_precision = precision_score(y_train, y_train_pred)
            
            # Predict and evaluate on testing data
            y_test_pred = xgb_classifier.predict(X_test)
            test_accuracy = accuracy_score(y_test, y_test_pred)
            test_f1 = f1_score(y_test, y_test_pred)
            test_recall = recall_score(y_test, y_test_pred)
            test_precision = precision_score(y_test, y_test_pred)
            
            # Calculate elapsed time
            end_time = time.time()
            elapsed_time = end_time - start_time
            
            # Append training and testing results separately, including parameter values and execution time
            params = f'LR: {lr}, Depth: {depth}, Estimators: {n_est}'
            
            print(params, elapsed_time)
            
            results.append({
                'Parameters': params,
                'Data Split': 'Training',
                'Accuracy': train_accuracy,
                'F1 Score': train_f1,
                'Recall': train_recall,
                'Precision': train_precision,
                'Execution Time (s)': elapsed_time
            })
            results.append({
                'Parameters': params,
                'Data Split': 'Testing',
                'Accuracy': test_accuracy,
                'F1 Score': test_f1,
                'Recall': test_recall,
                'Precision': test_precision,
                'Execution Time (s)': elapsed_time
            })

# Convert the list to a DataFrame
results_df = pd.DataFrame(results)

LR: 0.01, Depth: 3, Estimators: 20 2.4953768253326416
LR: 0.01, Depth: 3, Estimators: 50 3.1014468669891357
LR: 0.01, Depth: 3, Estimators: 100 4.545229911804199
LR: 0.01, Depth: 3, Estimators: 200 10.813835144042969
LR: 0.01, Depth: 3, Estimators: 500 17.317631483078003
LR: 0.01, Depth: 3, Estimators: 1000 33.41537570953369
LR: 0.01, Depth: 6, Estimators: 20 2.6912264823913574
LR: 0.01, Depth: 6, Estimators: 50 7.183120012283325
LR: 0.01, Depth: 6, Estimators: 100 5.082107782363892
LR: 0.01, Depth: 6, Estimators: 200 13.874131202697754
LR: 0.01, Depth: 6, Estimators: 500 32.64604997634888
LR: 0.01, Depth: 6, Estimators: 1000 49.293800592422485
LR: 0.01, Depth: 9, Estimators: 20 3.430760145187378
LR: 0.01, Depth: 9, Estimators: 50 9.819521188735962
LR: 0.01, Depth: 9, Estimators: 100 7.832666397094727
LR: 0.01, Depth: 9, Estimators: 200 16.661863803863525
LR: 0.01, Depth: 9, Estimators: 500 43.02814531326294
LR: 0.01, Depth: 9, Estimators: 1000 67.05780363082886
LR: 0.1, Depth: 3, Esti

In [11]:
results_df.head()

,Parameters,Data Split,Accuracy,F1 Score,Recall,Precision,Execution Time (s)
0,"LR: 0.01, Depth: 3, Estimators: 20",Training,0.948198,0.947371,0.932959,0.962236,2.495377
1,"LR: 0.01, Depth: 3, Estimators: 20",Testing,0.947620,0.946941,0.932972,0.961335,2.495377
2,"LR: 0.01, Depth: 3, Estimators: 50",Training,0.953933,0.952658,0.927452,0.979272,3.101447
3,"LR: 0.01, Depth: 3, Estimators: 50",Testing,0.953687,0.952531,0.927496,0.978956,3.101447
4,"LR: 0.01, Depth: 3, Estimators: 100",Training,0.951629,0.949752,0.914718,0.987576,4.545230


In [12]:
results_df.to_excel('data2_results/XGBoost_results.xlsx', index=False)

## AdaBoost

To experiment with the AdaBoost algorithm and tweak its parameters, we can adjust the number of estimators (n_estimators) and the learning rate (learning_rate). These are two key parameters for AdaBoost that influence the performance and complexity of the resulting model. The n_estimators parameter controls the number of sequential models to be added to correct the errors of the previous models, while learning_rate scales the contribution of each model.

In [13]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# Define ranges for the parameters to test
n_estimators_list = [10, 25, 50, 100] #, 500, 1000, 5000] #Too slow to run all. At 1000 took 4 hours and crashed.
learning_rates_list = [0.01, 0.1, 0.5, 1] #, 2]

# Initialize a list to store the results
results = []

# Loop over the combinations of n_estimators and learning_rates
for n_estimators in n_estimators_list:
    for learning_rate in learning_rates_list:
        start_time = time.time()  # Start timing
        
        # Instantiate and fit the AdaBoost classifier
        ada_classifier = AdaBoostClassifier(n_estimators=n_estimators, learning_rate=learning_rate)
        ada_classifier.fit(X_train, y_train)
        
        # Predict and evaluate on training data
        y_train_pred = ada_classifier.predict(X_train)
        train_accuracy = accuracy_score(y_train, y_train_pred)
        train_f1 = f1_score(y_train, y_train_pred)
        train_recall = recall_score(y_train, y_train_pred)
        train_precision = precision_score(y_train, y_train_pred)
        
        # Predict and evaluate on testing data
        y_test_pred = ada_classifier.predict(X_test)
        test_accuracy = accuracy_score(y_test, y_test_pred)
        test_f1 = f1_score(y_test, y_test_pred)
        test_recall = recall_score(y_test, y_test_pred)
        test_precision = precision_score(y_test, y_test_pred)
        
        # Calculate elapsed time
        end_time = time.time()
        elapsed_time = end_time - start_time
        
        # Append training and testing results separately, including parameter values and execution time
        params = f'Estimators: {n_estimators}, LR: {learning_rate}'
        print(params, elapsed_time)
        results.append({
            'Parameters': params,
            'Data Split': 'Training',
            'Accuracy': train_accuracy,
            'F1 Score': train_f1,
            'Recall': train_recall,
            'Precision': train_precision,
            'Execution Time (s)': elapsed_time
        })
        results.append({
            'Parameters': params,
            'Data Split': 'Testing',
            'Accuracy': test_accuracy,
            'F1 Score': test_f1,
            'Recall': test_recall,
            'Precision': test_precision,
            'Execution Time (s)': elapsed_time
        })

# Convert the list to a DataFrame
results_df = pd.DataFrame(results)

Estimators: 10, LR: 0.01 38.72339916229248
Estimators: 10, LR: 0.1 38.582165002822876
Estimators: 10, LR: 0.5 38.85813236236572
Estimators: 10, LR: 1 38.44630789756775
Estimators: 25, LR: 0.01 95.35388898849487
Estimators: 25, LR: 0.1 96.85238933563232
Estimators: 25, LR: 0.5 97.05669522285461
Estimators: 25, LR: 1 95.1578369140625
Estimators: 50, LR: 0.01 190.02480030059814
Estimators: 50, LR: 0.1 189.15585112571716
Estimators: 50, LR: 0.5 189.37187027931213
Estimators: 50, LR: 1 189.97536420822144
Estimators: 100, LR: 0.01 381.32956624031067
Estimators: 100, LR: 0.1 381.91789197921753
Estimators: 100, LR: 0.5 380.7166266441345
Estimators: 100, LR: 1 379.6542992591858


In [14]:
results_df.to_excel('data2_results/AdaBoost_results.xlsx', index=False)
results_df.head()

,Parameters,Data Split,Accuracy,F1 Score,Recall,Precision,Execution Time (s)
0,"Estimators: 10, LR: 0.01",Training,0.931711,0.928612,0.888748,0.972221,38.723399
1,"Estimators: 10, LR: 0.01",Testing,0.932162,0.929277,0.889603,0.972655,38.723399
2,"Estimators: 10, LR: 0.1",Training,0.937917,0.935077,0.894629,0.979357,38.582165
3,"Estimators: 10, LR: 0.1",Testing,0.938704,0.936100,0.896184,0.979738,38.582165
4,"Estimators: 10, LR: 0.5",Training,0.948706,0.947274,0.922015,0.973956,38.858132


## AdaBoost with Decision Tree clacifier

AdaBoost, as implemented in scikit-learn (AdaBoostClassifier), doesn't directly accept a max_depth parameter because AdaBoost can be used with various types of base estimators, not just decision trees. This is a frequent way of using AdaBoost.

In [18]:
import time
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# Define ranges for the parameters to test
n_estimators_list = [10, 25, 50] #, 500, 1000, 5000]
learning_rates_list = [0.01, 0.1, 0.3]
max_depths = [3, 5, 7]

# Initialize a list to store the results
results = []

# Loop over the combinations of n_estimators, learning_rates, and max_depths
for n_estimators in n_estimators_list:
    for learning_rate in learning_rates_list:
        for max_depth in max_depths:
            start_time = time.time()  # Start timing
            
            # Create a DecisionTreeClassifier with the specified max_depth
            base_estimator = DecisionTreeClassifier(max_depth=max_depth)
            
            # Instantiate and fit the AdaBoost classifier with the DecisionTree base estimator
            ada_classifier = AdaBoostClassifier(
                base_estimator=base_estimator,
                n_estimators=n_estimators,
                learning_rate=learning_rate
            )
            ada_classifier.fit(X_train, y_train)
            
            # Predict and evaluate on training data
            y_train_pred = ada_classifier.predict(X_train)
            train_accuracy = accuracy_score(y_train, y_train_pred)
            train_f1 = f1_score(y_train, y_train_pred)
            train_recall = recall_score(y_train, y_train_pred)
            train_precision = precision_score(y_train, y_train_pred)
            
            # Predict and evaluate on testing data
            y_test_pred = ada_classifier.predict(X_test)
            test_accuracy = accuracy_score(y_test, y_test_pred)
            test_f1 = f1_score(y_test, y_test_pred)
            test_recall = recall_score(y_test, y_test_pred)
            test_precision = precision_score(y_test, y_test_pred)
            
            # Calculate elapsed time
            end_time = time.time()
            elapsed_time = end_time - start_time
            
            # Append training and testing results separately, including parameter values and execution time
            params = f'Estimators: {n_estimators}, LR: {learning_rate}, Depth: {max_depth}'
            print(params, elapsed_time)
            results.append({
                'Parameters': params,
                'Data Split': 'Training',
                'Accuracy': train_accuracy,
                'F1 Score': train_f1,
                'Recall': train_recall,
                'Precision': train_precision,
                'Execution Time (s)': elapsed_time
            })
            results.append({
                'Parameters': params,
                'Data Split': 'Testing',
                'Accuracy': test_accuracy,
                'F1 Score': test_f1,
                'Recall': test_recall,
                'Precision': test_precision,
                'Execution Time (s)': elapsed_time
            })

# Convert the list to a DataFrame
results_df = pd.DataFrame(results)


Estimators: 10, LR: 0.01, Depth: 3 102.31569004058838
Estimators: 10, LR: 0.01, Depth: 5 166.09842157363892
Estimators: 10, LR: 0.01, Depth: 7 233.21162915229797
Estimators: 10, LR: 0.1, Depth: 3 104.77864599227905
Estimators: 10, LR: 0.1, Depth: 5 173.04592084884644
Estimators: 10, LR: 0.1, Depth: 7 233.71685528755188
Estimators: 10, LR: 0.3, Depth: 3 102.45896935462952
Estimators: 10, LR: 0.3, Depth: 5 167.55185723304749
Estimators: 10, LR: 0.3, Depth: 7 227.46368050575256
Estimators: 25, LR: 0.01, Depth: 3 261.5536754131317
Estimators: 25, LR: 0.01, Depth: 5 412.54430961608887
Estimators: 25, LR: 0.01, Depth: 7 579.35755443573
Estimators: 25, LR: 0.1, Depth: 3 257.77327489852905
Estimators: 25, LR: 0.1, Depth: 5 414.39040422439575
Estimators: 25, LR: 0.1, Depth: 7 569.3035671710968
Estimators: 25, LR: 0.3, Depth: 3 267.8144872188568
Estimators: 25, LR: 0.3, Depth: 5 422.5403587818146
Estimators: 25, LR: 0.3, Depth: 7 578.2721581459045
Estimators: 50, LR: 0.01, Depth: 3 517.151166677

In [19]:
results_df.to_excel('data2_results/AdaBoostDT_results.xlsx', index=False)
results_df.head()

,Parameters,Data Split,Accuracy,F1 Score,Recall,Precision,Execution Time (s)
0,"Estimators: 10, LR: 0.01, Depth: 3",Training,0.947061,0.945133,0.912360,0.980347,102.315690
1,"Estimators: 10, LR: 0.01, Depth: 3",Testing,0.947980,0.946304,0.914947,0.979887,102.315690
2,"Estimators: 10, LR: 0.01, Depth: 5",Training,0.965819,0.965338,0.952397,0.978634,166.098422
3,"Estimators: 10, LR: 0.01, Depth: 5",Testing,0.965329,0.964961,0.952928,0.977302,166.098422
4,"Estimators: 10, LR: 0.01, Depth: 7",Training,0.977780,0.977484,0.965136,0.990153,233.211629


## Gradient Boosting Machine

To perform a similar parameter tweaking experiment with a Gradient Boosting Machine (GBM) using scikit-learn's GradientBoostingClassifier, we can iterate over combinations of n_estimators, learning_rate, and max_depth. These parameters are analogous to those adjusted in the AdaBoost and XGBoost, serving similar purposes:

n_estimators: controls the number of sequential trees to be modeled.
learning_rate: scales the contribution of each tree.
max_depth: sets the maximum depth of each tree.

In [ ]:
import time
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# Define ranges for the parameters to test
n_estimators_list = [10, 20, 50]
learning_rates_list = [0.01, 0.1, 0.3]
max_depths = [3, 5, 7]

# Initialize a list to store the results
results = []

# Loop over the combinations of n_estimators, learning_rates, and max_depths
for n_estimators in n_estimators_list:
    for learning_rate in learning_rates_list:
        for max_depth in max_depths:
            start_time = time.time()  # Start timing
            
            # Instantiate and fit the GradientBoostingClassifier
            gbm_classifier = GradientBoostingClassifier(
                n_estimators=n_estimators,
                learning_rate=learning_rate,
                max_depth=max_depth
            )
            gbm_classifier.fit(X_train, y_train)
            
            # Predict and evaluate on training data
            y_train_pred = gbm_classifier.predict(X_train)
            train_accuracy = accuracy_score(y_train, y_train_pred)
            train_f1 = f1_score(y_train, y_train_pred)
            train_recall = recall_score(y_train, y_train_pred)
            train_precision = precision_score(y_train, y_train_pred)
            
            # Predict and evaluate on testing data
            y_test_pred = gbm_classifier.predict(X_test)
            test_accuracy = accuracy_score(y_test, y_test_pred)
            test_f1 = f1_score(y_test, y_test_pred)
            test_recall = recall_score(y_test, y_test_pred)
            test_precision = precision_score(y_test, y_test_pred)
            
            # Calculate elapsed time
            end_time = time.time()
            elapsed_time = end_time - start_time
            
            # Append training and testing results separately, including parameter values and execution time
            params = f'Estimators: {n_estimators}, LR: {learning_rate}, Depth: {max_depth}'
            print(params, elapsed_time)
            results.append({
                'Parameters': params,
                'Data Split': 'Training',
                'Accuracy': train_accuracy,
                'F1 Score': train_f1,
                'Recall': train_recall,
                'Precision': train_precision,
                'Execution Time (s)': elapsed_time
            })
            results.append({
                'Parameters': params,
                'Data Split': 'Testing',
                'Accuracy': test_accuracy,
                'F1 Score': test_f1,
                'Recall': test_recall,
                'Precision': test_precision,
                'Execution Time (s)': elapsed_time
            })

# Convert the list to a DataFrame
results_df = pd.DataFrame(results)

Estimators: 10, LR: 0.01, Depth: 3 548.9556698799133
Estimators: 10, LR: 0.01, Depth: 5 633.0351781845093
Estimators: 10, LR: 0.01, Depth: 7 760.003781080246
Estimators: 10, LR: 0.1, Depth: 3 221.40730714797974
Estimators: 10, LR: 0.1, Depth: 5 349.8702447414398
Estimators: 10, LR: 0.1, Depth: 7 564.2601475715637
Estimators: 10, LR: 0.3, Depth: 3 339.71960520744324
Estimators: 10, LR: 0.3, Depth: 5 462.6554033756256
Estimators: 10, LR: 0.3, Depth: 7 454.9595196247101
Estimators: 20, LR: 0.01, Depth: 3 407.6106336116791
Estimators: 20, LR: 0.01, Depth: 5 680.3393642902374
Estimators: 20, LR: 0.01, Depth: 7 2105.3441553115845
Estimators: 20, LR: 0.1, Depth: 3 400.140291929245


In [ ]:
results_df.to_excel('data2_results/GBM_results.xlsx', index=False)
results_df.head()